In [14]:
import sys
import os
niftynet_path = '/home/tom/phd/NiftyNet-Generator-PR/NiftyNet'
sys.path.append(niftynet_path)
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from niftynet.io.image_reader import ImageReader
from niftynet.io.image_sets_partitioner import ImageSetsPartitioner
from collections import namedtuple

from niftynet.contrib.preprocessors.preprocessing import Preprocessing
from niftynet.contrib.csv_reader.sampler_csv_rows import ImageWindowDatasetCSV
from niftynet.contrib.csv_reader.sampler_resize_v2_csv import ResizeSamplerCSV as ResizeSampler
from niftynet.contrib.csv_reader.csv_reader import CSVReader

In [15]:
#### Some setup
NetParam = namedtuple('NetParam', 'normalise_foreground_only foreground_type multimod_foreground_type histogram_ref_file norm_type cutoff normalisation whitening')
ActionParam = namedtuple('ActionParam', 'random_flipping_axes scaling_percentage rotation_angle rotation_angle_x rotation_angle_y rotation_angle_z do_elastic_deformation num_ctrl_points deformation_sigma proportion_to_deform')
class TaskParam:
    def __init__(self, classes):
        self.image = classes
net_param = NetParam(normalise_foreground_only=False, foreground_type='threshold_plus', multimod_foreground_type = 'and', histogram_ref_file='mapping.txt', norm_type='percentile', cutoff=(0.05, 0.95), normalisation=False, whitening=True)
action_param = ActionParam(random_flipping_axes=[], scaling_percentage=[], rotation_angle=None, rotation_angle_x=None, rotation_angle_y=None, rotation_angle_z=None, do_elastic_deformation=False, num_ctrl_points=6, deformation_sigma=50, proportion_to_deform=0.9)

### 1) Create a csv of labels and show how it can be returned by the CSV Reader


In [16]:
from niftynet.utilities.download import download
download('mr_ct_regression_model_zoo_data')
labels_location = 'ct.csv'
files = [file for file in os.listdir('/home/tom/niftynet/data/mr_ct_regression/CT_zero_mean') if file.endswith('.nii.gz')]
pd.DataFrame(data=[(file.replace('.nii.gz', ''), file.replace('.nii.gz', '')) for file in files]).to_csv('label.csv', index=None, header=['subject_id', 'label'])
pd.read_csv('label.csv')

Accessing: https://github.com/NifTK/NiftyNetModelZoo
mr_ct_regression_model_zoo_data: FAIL. 
No NiftyNet example was found for mr_ct_regression_model_zoo_data.


,subject_id,label
0,RAN,RAN
1,HAL,HAL
2,MIL,MIL
3,CHA,CHA
4,GRA,GRA
5,PLA,PLA
6,NAR,NAR
7,WEB,WEB
8,PAR,PAR
9,HON,HON


In [17]:
#### Testing the CSV Reader on labels
# Make sure we accept 'Label', 'label', 'LABEL'
task_param = TaskParam(['image'])
image_data_param = {'CT': {'path_to_search': '~/niftynet/data/mr_ct_regression/CT_zero_mean', 'filename_contains': 'nii'}}
#csv_data_file is a csv with data
csv_data_param = {'label': {'csv_data_file': 'label.csv', 'to_ohe': True}}
grouping_param = {'image': (['CT'])}

image_sets_partitioner = ImageSetsPartitioner().initialise(image_data_param)
image_reader = ImageReader().initialise(image_data_param, grouping_param, file_list=image_sets_partitioner.all_files)
preprocessing = Preprocessing(net_param, action_param, task_param)
normalisation_layers = preprocessing.prepare_normalisation_layers()
augmentation_layers = preprocessing.prepare_augmentation_layers()
image_reader.add_preprocessing_layers(normalisation_layers + augmentation_layers)
csv_reader = CSVReader(('label',)).initialise(csv_data_param, {'label': (['label'])}, file_list=image_sets_partitioner.all_files)
print('One sample from the csv_reader:', np.squeeze(csv_reader(idx=13)[1]['label']))
window_sizes = {'image': (100, 100, 1), 'label': (1, 1, 1)}
sampler = ResizeSampler(reader=image_reader,
                        csv_reader=csv_reader,
                        window_sizes=window_sizes,
                        num_threads=2,
                        smaller_final_batch_mode='drop',
                        batch_size=2,
                        queue_length=2)
sample = next(sampler())
print(sample['image'].shape)
print(sample['label'].shape)

INFO:niftynet: 

Number of subjects 15, input section names: ['subject_id', 'CT']
-- using all subjects (without data partitioning).

INFO:niftynet: Image reader: loading 15 subjects from sections ['CT'] as input [image]
One sample from the csv_reader: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
INFO:niftynet: reading size of preprocessed images
(1, 100, 100, 1, 1, 1)
(1, 15, 1, 1, 1, 1)


### 2) Create a csv of features and show how it can be returned by the CSV Reader


In [18]:
from niftynet.utilities.download import download
download('mr_ct_regression_model_zoo_data')
labels_location = 'ct.csv'
files = [file.replace('.nii.gz', '') for file in os.listdir('/home/tom/niftynet/data/mr_ct_regression/CT_zero_mean') if file.endswith('.nii.gz')]

pd.DataFrame(data=[tuple([file] + list(np.random.randn(10))) for file in files]).to_csv('features.csv', index=None, header=['subject_id'] + [str(x) for x in range(10)])
pd.read_csv('features.csv')

Accessing: https://github.com/NifTK/NiftyNetModelZoo
mr_ct_regression_model_zoo_data: FAIL. 
No NiftyNet example was found for mr_ct_regression_model_zoo_data.


,subject_id,0,1,2,3,4,5,6,7,8,9
0,RAN,-2.018905,0.016700,0.753500,0.225668,-0.453137,0.007740,0.405050,-0.038025,-1.144459,-0.437604
1,HAL,0.750608,1.339236,0.393103,0.403952,0.034992,1.743924,0.222566,0.411477,-0.557221,0.407324
2,MIL,-0.581589,-0.988355,-1.536794,0.344513,-0.161370,-0.118697,-0.322209,0.063637,-1.028091,-0.392706
3,CHA,2.234084,-0.264239,-0.923163,-0.817617,0.388325,0.560078,-0.477164,1.966860,-0.018510,0.442490
4,GRA,0.653114,0.477493,0.650812,1.353049,0.734319,-1.539332,0.653350,-0.752011,-0.634769,-0.231503
5,PLA,-0.983351,-1.600570,0.223609,-1.100547,-0.430255,1.315413,-0.702730,-1.838355,-0.810046,2.419473
6,NAR,0.810422,-2.304446,-0.079281,0.456476,0.643375,0.268823,0.366755,0.530736,1.345358,0.515989
7,WEB,0.710464,-0.665521,-1.130775,-0.437872,-0.359845,-0.128057,-0.472871,-0.060746,1.754929,-1.009953
8,PAR,1.380408,-0.825570,-0.630116,-2.312509,-0.335770,-1.176994,-0.455428,-0.757432,-0.171152,0.803127
9,HON,-0.312656,-0.707139,0.571808,0.682854,0.907817,0.067288,-0.384104,-1.030702,0.015677,-0.808565


In [19]:
task_param = TaskParam(['image'])
image_data_param = {'CT': {'path_to_search': '~/niftynet/data/mr_ct_regression/CT_zero_mean', 'filename_contains': 'nii'}}
csv_data_param = {'features': {'csv_data_file': 'features.csv', 'to_ohe': False}}
grouping_param = {'image': (['CT'])}
image_sets_partitioner = ImageSetsPartitioner().initialise(image_data_param)
image_reader = ImageReader().initialise(image_data_param, grouping_param, file_list=image_sets_partitioner.all_files)
preprocessing = Preprocessing(net_param, action_param, task_param)
normalisation_layers = preprocessing.prepare_normalisation_layers()
augmentation_layers = preprocessing.prepare_augmentation_layers()
image_reader.add_preprocessing_layers(normalisation_layers + augmentation_layers)
csv_reader = CSVReader(('features',)).initialise(csv_data_param, {'features': ['features']}, file_list=image_sets_partitioner.all_files)
print('One sample from the csv_reader:', np.squeeze(csv_reader(idx=13)[1]['features']))
window_sizes = {'image': (100, 100, 1), 'features': (1, 1, 1)}
sampler = ResizeSampler(reader=image_reader,
                        csv_reader=csv_reader,
                        window_sizes=window_sizes,
                        num_threads=2,
                        smaller_final_batch_mode='drop',
                        batch_size=2,
                        queue_length=2)
sample = next(sampler())
print(sample['image'].shape)
print(sample['features'].shape)
print(sample.keys())

INFO:niftynet: 

Number of subjects 15, input section names: ['subject_id', 'CT']
-- using all subjects (without data partitioning).

INFO:niftynet: Image reader: loading 15 subjects from sections ['CT'] as input [image]
One sample from the csv_reader: [ 0.23841972 -0.96603888 -0.07318273  1.59560139 -0.09326917 -1.04853203
 -0.65709902 -1.17890471  0.7956195  -0.97414747]
INFO:niftynet: reading size of preprocessed images
(1, 100, 100, 1, 1, 1)
(1, 10, 1, 1, 1, 1)
dict_keys(['image_location', 'image', 'features', 'features_location'])


# Testing the CSV Reader on labels AND Features



In [20]:
# Make sure we accept 'Label', 'label', 'LABEL'
task_param = TaskParam(['image'])
image_data_param = {'CT': {'path_to_search': '~/niftynet/data/mr_ct_regression/CT_zero_mean', 'filename_contains': 'nii'}}
csv_data_param = {'label': {'csv_data_file': 'label.csv', 'to_ohe': True},
                  'features': {'csv_data_file': 'features.csv', 'to_ohe': False}}
grouping_param = {'image': (['CT'])}

image_sets_partitioner = ImageSetsPartitioner().initialise(image_data_param)
image_reader = ImageReader().initialise(image_data_param, grouping_param, file_list=image_sets_partitioner.all_files)
preprocessing = Preprocessing(net_param, action_param, task_param)
normalisation_layers = preprocessing.prepare_normalisation_layers()
augmentation_layers = preprocessing.prepare_augmentation_layers()
image_reader.add_preprocessing_layers(normalisation_layers + augmentation_layers)

csv_reader = CSVReader(('label', 'features')).initialise(csv_data_param,
                                                         {'label': (['label']), 'features': (['features'])},
                                                         file_list=image_sets_partitioner.all_files)


print('One sample from the csv_reader:', np.squeeze(csv_reader(idx=13)[1]['label']))
window_sizes = {'image': (100, 100, 1), 'label': (1, 1, 1)}
sampler = ResizeSampler(reader=image_reader,
                        csv_reader=csv_reader,
                        window_sizes=window_sizes,
                        num_threads=2,
                        smaller_final_batch_mode='drop',
                        batch_size=2,
                        queue_length=2)
sample = next(sampler())
print(sample['image'].shape)
print(sample['label'].shape)
print(sample['features'].shape)


INFO:niftynet: 

Number of subjects 15, input section names: ['subject_id', 'CT']
-- using all subjects (without data partitioning).

INFO:niftynet: Image reader: loading 15 subjects from sections ['CT'] as input [image]
One sample from the csv_reader: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
INFO:niftynet: reading size of preprocessed images
(1, 100, 100, 1, 1, 1)
(1, 15, 1, 1, 1, 1)
(1, 10, 1, 1, 1, 1)


In [30]:
modalities = ['t1ce.', 't1.', 'flair.', 't2.']
def get_modality(string):
    return modalities[[True if mod in string else False for mod in modalities].index(True)][:-1]
    
files = [(file.replace('.nii.gz', ''), get_modality(file)) \
         for file in os.listdir('/home/tom/data/BRATS_18_SPLITS/train') if 'seg' not in file]
pd.DataFrame(data=files, columns=['subject_id', 'label']).to_csv('/home/tom/phd/NiftyNet-Generator-PR/NiftyNet/modality_labels.csv', index=None)
